In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display, Image, HTML
import tensorflow as tf
import cv2
from time import strftime

TRAIN_DIR = '../input/train/'
TEST_DIR = '../input/test/'

In [ ]:
# used for scaling/normalization
IMAGE_SIZE = 96; # 150x150.  Also, 224, 96, 64, and 32 are also common
CHANNELS = 3
pixel_depth = 255.0  # Number of levels per pixel.

TRAINING_AND_VALIDATION_SIZE_DOGS = 1000 
TRAINING_AND_VALIDATION_SIZE_CATS = 1000 
TRAINING_AND_VALIDATION_SIZE_ALL  = 2000
TRAINING_SIZE = 1600  # TRAINING_SIZE + VALID_SIZE must equal TRAINING_AND_VALIDATION_SIZE_ALL
VALID_SIZE = 400
TEST_SIZE_ALL = 500

if (TRAINING_SIZE + VALID_SIZE != TRAINING_AND_VALIDATION_SIZE_ALL):
    print ("Error, check that TRAINING_SIZE+VALID_SIZE is equal to TRAINING_AND_VALIDATION_SIZE_ALL")
    exit ()

train_images_path = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] 
train_dogs =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats =   [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR) if 'cat' in i]
test_images_path =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

train_images_path = train_dogs[:TRAINING_AND_VALIDATION_SIZE_DOGS] + train_cats[:TRAINING_AND_VALIDATION_SIZE_CATS]
train_labels = np.array (([[1, 0]] * TRAINING_AND_VALIDATION_SIZE_DOGS) + ([[0, 1]] * TRAINING_AND_VALIDATION_SIZE_CATS))
test_images_path =  test_images_path[:TEST_SIZE_ALL]

train_images_path = np.array(train_images_path)
test_images_path = np.array(test_images_path)

In [ ]:
# resizes to IMAGE_SIZE/IMAGE_SIZE while keeping aspect ratio the same.  pads on right/bottom as appropriate 
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR) #cv2.IMREAD_GRAYSCALE
    if (img.shape[0] >= img.shape[1]): # height is greater than width
        resizeto = (IMAGE_SIZE, int (round (IMAGE_SIZE * (float (img.shape[1])  / img.shape[0]))));
    else:
        resizeto = (int (round (IMAGE_SIZE * (float (img.shape[0])  / img.shape[1]))), IMAGE_SIZE);
    
    resized = cv2.resize(img, (resizeto[1], resizeto[0]), interpolation=cv2.INTER_CUBIC)
    padded = cv2.copyMakeBorder(resized, 0, IMAGE_SIZE - resized.shape[0], 0, IMAGE_SIZE - resized.shape[1], cv2.BORDER_CONSTANT, 0)
        
    return padded

def randomize(dataset, labels):
    assert len(dataset) == len(labels)
    p = np.random.permutation(len(dataset))
    shuffled_dataset = dataset[p]
    shuffled_labels = labels[p]
    
randomize(train_images_path, train_labels)

train_images = np.array([read_image(path) for path in train_images_path])
test_images = np.array([read_image(path) for path in test_images_path])

valid_images = train_images[:VALID_SIZE]
valid_labels = train_labels[:VALID_SIZE]
train_images = train_images[VALID_SIZE:VALID_SIZE+TRAINING_SIZE]
train_labels  = train_labels[VALID_SIZE:VALID_SIZE+TRAINING_SIZE]

plt.imshow(cv2.cvtColor(train_images[0], cv2.COLOR_BGR2RGB))
print(train_labels[0])

In [ ]:
def flatten_cnn(layer):
    layer_shape = layer.get_shape().as_list()
    n_out = layer_shape[1] * layer_shape[2] * layer_shape[3]
    return tf.reshape(layer, [-1, n_out])

def build_nn(shape, X, name):
    n_before = int(X.get_shape()[1])
    W = tf.Variable(tf.truncated_normal([n_before, shape], stddev=0.1), name=name+"_W")
    b = tf.Variable(tf.constant(0.1, shape=[shape]), name=name+"_b")
    layer = tf.matmul(X, W)+b
    return layer

def build_cnn(cnn_shape, patch_shape, X, name, stride=1):
    n_before = int(X.get_shape()[3])
    W = tf.Variable(tf.truncated_normal([patch_shape[0], patch_shape[1], n_before, cnn_shape], stddev=0.1),
                   name=name+"_W")
    b = tf.Variable(tf.constant(0.1, shape=[cnn_shape]), name=name+"_b")
    layer = tf.nn.conv2d(X, W, strides=[1, stride, stride, 1], padding='SAME') + b
    return layer

def build_cnn_relu(cnn_shape, patch_shape, X, name, stride=1):
    layer = tf.nn.relu(build_cnn(cnn_shape, patch_shape, X, name, stride))
    return layer

def max2d_pool(layer):
    return tf.nn.max_pool(layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# http://r2rt.com/implementing-batch-normalization-in-tensorflow.html
def batch_norm(layer, is_training, name, decay = 0.999, does_scale=True):
    layer_shape = layer.get_shape().as_list()
    depth_dim = len(layer_shape)-1
    depth = layer_shape[-1]
    if does_scale:
        scale = tf.Variable(tf.ones(depth), name=name+"_BN_s")
    else:
        scale = None
    beta = tf.Variable(tf.zeros(depth), name=name+"_BN_b")
    mov_mean = tf.Variable(tf.zeros(depth), trainable=False, name=name+"_BN_pm")
    mov_var = tf.Variable(tf.ones(depth), trainable=False, name=name+"_BN_pv")
    
    def use_batch_with_update_mov():
        batch_mean, batch_var = tf.nn.moments(layer,list(range(depth_dim)))
        train_mean = tf.assign(mov_mean,
                               mov_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(mov_var,
                              mov_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(layer,
                batch_mean, batch_var, beta, scale, 0.001)
        
    def use_mov():
        return tf.nn.batch_normalization(layer,
            mov_mean, mov_var, beta, scale, 0.001)
    
    return tf.cond(is_training, use_batch_with_update_mov, use_mov)
    
def build_nn_bn(shape, X, is_training, name, decay = 0.999, does_scale=True):
    n_before = int(X.get_shape()[1])
    W = tf.Variable(tf.truncated_normal([n_before, shape], stddev=0.1), name=name+"_W")
    layer = tf.matmul(X, W)
    return batch_norm(layer, is_training, name, decay, does_scale)

def build_nn_bn_relu(shape, X, is_training, name, decay = 0.999):
    layer = tf.nn.relu(build_nn_bn(shape, X, is_training, name, decay, False))
    return layer
    
def build_cnn_bn(cnn_shape, patch_shape, X, is_training, name, stride=1, decay=0.999, does_scale=True):
    n_before = int(X.get_shape()[3])
    W = tf.Variable(tf.truncated_normal([patch_shape[0], patch_shape[1], n_before, cnn_shape], stddev=0.1),
                   name=name+"_W")
    layer = tf.nn.conv2d(X, W, strides=[1, stride, stride, 1], padding='SAME')
    return batch_norm(layer, is_training, name, decay, does_scale)

def build_cnn_bn_relu(cnn_shape, patch_shape, X, is_training, name, stride=1, decay=0.999):
    layer = tf.nn.relu(build_cnn_bn(cnn_shape, patch_shape, X, is_training, name, stride, decay, False))
    return layer

def slice_label(tf_label, len_tuple):
    cur = 0
    sliced = []
    for l in len_tuple:
        sliced.append(tf.slice(tf_label, [0, cur], [-1, l]))
        cur += l
    return tuple(sliced)


In [ ]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 96, 96, 3])
is_training = tf.placeholder(tf.bool)

# Small inception model
# http://laonple.blog.me/220704822964
cnn_1_5 = build_cnn_bn_relu(12, [5,5], X, is_training, "cnn_1_5")
cnn_1_3 = build_cnn_bn_relu(36, [3,3], X, is_training, "cnn_1_3")
cnn_1_concat = tf.concat([cnn_1_5, cnn_1_3], 3)
cnn_1_pool = max2d_pool(cnn_1_concat) # 48 * 48 * 48

cnn_2_5 = build_cnn_bn_relu(18, [5,5], cnn_1_pool, is_training, "cnn_2_5")
cnn_2_3 = build_cnn_bn_relu(48, [3,3], cnn_1_pool, is_training, "cnn_2_3")
cnn_2_1 = build_cnn_bn_relu(30, [1,1], cnn_1_pool, is_training, "cnn_2_1")
cnn_2_concat = tf.concat([cnn_2_5, cnn_2_3, cnn_2_1], 3)
cnn_2_pool = max2d_pool(cnn_2_concat) # 24 * 24 * 96

cnn_3_3_reduce = build_cnn_bn_relu(32, [1,1], cnn_2_pool, is_training, "cnn_3_3_reduce")
cnn_3_3 = build_cnn_bn_relu(48, [3,3], cnn_3_3_reduce, is_training, "cnn_3_3")
cnn_3_1 = build_cnn_bn_relu(16, [1,1], cnn_2_pool, is_training, "cnn_3_1")
cnn_3_concat = tf.concat([cnn_3_3, cnn_3_1], 3)
cnn_3_pool = max2d_pool(cnn_3_concat) # 12 * 12 * 64

dense_1 = build_nn_bn_relu(1024, flatten_cnn(cnn_3_pool), is_training, "dense_1")

logit = build_nn(2, dense_1, "logit")
hyp = tf.nn.softmax(logit)

In [ ]:
Y = tf.placeholder(tf.float32, [None, 2])

learning_rate = tf.placeholder(tf.float32)
cost = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y)
cost_mean = tf.reduce_mean(cost) # mean of batch set

train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct = tf.equal(tf.argmax(Y,1), tf.argmax(hyp,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

def get_now_str():
    return strftime("%Y-%m-%d %H:%M:%S")

def do_train(max_epoch=4, batchsize=200, lr_init = 0.003, prog=False, stat=True):
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    trainsize = train_images.size
    batch_per_epoch = int(trainsize/batchsize)
    print ("[%s] Training %d, mini-batch %d * %d" % (get_now_str(), trainsize, batchsize, batch_per_epoch))
    epoch = 0

    i = 0
    num_trained = 0
    lr = lr_init
    
    train_index = 0
    
    while (epoch < max_epoch):
        batch_x = train_images[train_index:batchsize+train_index]
        batch_y = train_labels[train_index:batchsize+train_index]
        train_index += batchsize

        i += 1
        num_trained += batch_x.shape[0]

        cur_cost = sess.run((train, cost_mean),
                            feed_dict={X:batch_x, Y:batch_y, is_training: True, learning_rate:lr})[1]

        if i % 10 == 0 :
            if prog == True:
                print("                                        \r", end="")
            if stat == True :
                cv_cost, cv_acc = get_mean_in_batch(sess, (cost_mean, accuracy), cvimg, cvlabel, executor)
                cur_cost_test, cur_acc = sess.run((cost_mean, accuracy), feed_dict={X:batch_x, Y:batch_y, is_training: False})
                print ("[%s] %5.2f %4.2e %4.3f %4.2e %4.3f %3.2e" %
                    (get_now_str(), num_trained/trainsize, cur_cost_test, cur_acc, cv_cost, cv_acc, lr))
            else :
                print ("[%s] %4.2f %4.2e" % (get_now_str(), num_trained/trainsize, cur_cost))
        if prog == True:
            print ("%dth... lr = %.2e, cost = %.2e\r" % (i, lr, cur_cost), end="")
        lr = lr * (1 - 0.0003)
    print("                                        \r", end="")
    print("[%s] train complete" % get_now_str())
    print("test accuracy ---")
    print_accuracy(sess, testimg, testlabel, True, executor)
    print("train accuracy ---")
    print_accuracy(sess, trainimg, trainlabel, True, executor)

In [ ]:
do_train()